## Skip quardtree

**skip quardtree** - структура данных, позволяющая хранить множество точек и производить над ним операции, такие как локализация, вставка и удаление. Здесь мы рассмотри одну из реализаций - **randomized skip quardtree**.

Построим **randomized skip quardtree** для заданного множества точек $S$:

Рассмотрим последовательность вложенных подмножеств $\{S_i\}$ множества $S$. Пусть $S_0 = S$; $S_i$ - подмножество $S_{i-1}$, причем каждый элемент из $S_{i-1}$ входит в $S_{i}$ с вероятностью $p \in (0, 1)$. **randomized skip quadtree** для множества $S$ будет состоять из набора $\{Q_i\}$, где $Q_i$ — сжатое квадродерево над множеством $S_i$. Будем называть эти квадродеревья уровнями, при этом нулевой уровень содержит все точки из $S$.

Также мы должны уметь из верншины с i-го уровня перемещаться на соотвестсвущую ей вершину на i-1-ом уровне. Это можно делать несколькими способами:
    
* Просто хранить ссылку внутри вершины.
* Каждый уровень будет хранить ассоциативный массив из масок в вершины. Маску для вершину можно однозначно задать по её коордианатам.

### Локализация

Локализация происходит так же как и в обычных **quardtree**.
Сначала локализуемся в наивысшем уровне, начиная с `root`-вершины. Из полученной вершины переходим в соотвествующую ей вершину на предыдущем уровне. Снова запускаем локализацию, но теперь не из `root`'а, а из вершины, полученной предыдущей локализацией. Повотряем процесс пока не локализуемся в нулевом слое.

Идеологически это похоже на **skip list**: мы пользуемся тем, что локализация в **quardtree** происходит за линейное время (чем меньше точек в квадродереве тем, быстрее проходит операция), и сначала пытаемся локализоваться на том уровне, где меньше всего точек, а потом, при необходимости, спускаемся на уровнень ниже, и продолжаем операцию. Ниже показано, что локализация подобным образом имеет ассимптотику $O(\log n)$.

### Вставка

1. Локализация. В процессе локализации будем для каждого уровня запоминать вершину, на которой остановилась локализация внутри этого уровня.
2. Вставка точки в нулевой слой, в вершину, полученную локализацией из п.1.
3. С вероятностью $p$, переходим на уровень выше к вершине, которую мы запомнили в п.1, вставляем точку в эту вершину.
4. В случае удачи повторяем п.3, в случае неудачи завершаем операцию.
5. Если мы хотим перейти к слою которого нет, создаем новый слой, вствляем туда новую точку и завершаем операцию.

### Удаление

Запускаем локализацию. В процессе локализации будем удалять точку на всех уровнях, где она есть. Если на уровне стало ноль точек - удаляем уровень.

### Оценка времени работы
Такой сложный способ хранения множества точек дает нам бонус в скорости выполнения операций над ним. Из лемм ниже следует что время выполнения локализации оценивается в $O(\log n)$. Вставка и удаление также работает за $O(\log n)$, т.к. на каждом слое происходит не более $O(1)$ операций (добавление / удаление вершины в квадродерево).

**Лемма (О количестве шагов локализации на одном уровне):**
На каждом уровне совершается $O(1)$ шагов локализации для любой точки `x`.

**Док-во:**
Пусть в на `i`-ом уровне поиск точки `x`, начинающийся с корня, проходит по квадратам $p_0, p_1 \dots p_m$. Пусть случайная величина $j$ — количество шагов локализации внутри $Q_i$, тогда $p_{m - j}$ — последний квадрат из $p_0, p_1 \dots p_m$, являющийся интересным в $Q_{i + 1}$.

Оценим мат.ожидание случайной величины $j$. Пусть $T$ - множество встреченных на пути $p_{m - j + 1} \dots p_m$ непустых четвертинок. Чтобы $p_{m - j}$ был последним из $p_0, p_1 \dots p_m$ интересным квадратом в $Q_{i + 1}$ небходимо, чтобы среди $T$ только одна (вероятность этого назовём $Pr_1$) или ноль (вероятность этого назовём $Pr_0$) были непустыми в $Q_{i + 1}$. Иначе, если будет хотя бы пара непустых четвертинок, то их наименьший общий предок в дереве будет интересным квадратом и будет находиться глубже $p_{m - j}$. Таким образом, искомая вероятность не превосходит $Pr_0 + Pr_1$.

Пусть $|T| = t$, $q = 1 - p$.

$Pr_0 = q^t$, потому что $Pr_0$ означает то, что ни одна точка из $T$ не попала на уровень выше.

$Pr_1 = t \cdot pq^{t-1}$, потому что $Pr_1$ означает то, что ровна одна точка из $T$ не попала на уровень выше.

$Ej = \sum\limits_{j = 1}^{m} j \cdot Pr(j) = \sum\limits_{j = 1}^{m} j (q^t + t \cdot pq^{t-1}) = \sum\limits_{j = 1}^{m} j ((1-t) \cdot q^t + t \cdot q^{t-1}) $

В интересной вершине как минимум 2 непустые четверти (в одну из которых мы переходим), всего на пути $j$ интересных вершин, следственно, количество вершин в $T$ - хотя бы j. Следственно, количество точек в $T$ - тоже хотя бы j.

$ Ej                                                                         
\leq \sum\limits_{j = 1}^{m} j ((1-j) \cdot q^j + t \cdot q^{j-1})
<    \sum\limits_{j = 1}^{\infty} j ((1-j) \cdot q^j + j \cdot q^{j-1})
<    \sum\limits_{j = 1}^{\infty}( 2 j (j-1) \cdot q^{j-1} - j (j-1) \cdot q^j)
<    \sum\limits_{j = 1}^{\infty} 2 j (j-1) \cdot q^{j-1}
=    2 \sum\limits_{j = 1}^{\infty} (q^{j+1})''
=    2 (\sum\limits_{j = 1}^{\infty} q^{j+1})''
=    (2 q^2 / (1 - q))''
=    2p^{-3}
$

Получаем, $Ej < 2p^{-3}$. Следственно, $Ej = O(1)$. 

**Лемма (О количестве уровней):** 
Количество уровней - $O(\log n)$

**Док-во:**
Пусть $h$ - cлучайная величина, обозначающая количество уровней.

Оценим $p(h = k)$. 

$p(h = k) = p(h < k + 1) - p (h < k)$

$p(h < k) = (1 - p^k)^n$, потому что вероятность того, что при добавлении точка дойдет до уровня $k$ равняется $p^k$.

$p(h = k) = (1 - p^{k + 1})^n - (1 - p^k)^n < 1 - (1 - p^k)^n < np^k$

Оценим мат.ожидание $h$:

$ Eh
= \sum\limits_{k = 1}^{\infty}k \cdot p(h = k)
= \sum\limits_{k = 1}^{\log_{1/p}(n)}k \cdot p(h = k)
+ \sum\limits_{k = \log_{1/p}(n) + 1}^{\infty}k \cdot p(h = k)
$

Первая сумма:
$ \sum\limits_{k = 1}^{\log_{1/p}(n)}k \cdot p(h = k) 
< \sum\limits_{k = 1}^{\log_{1/p}(n)}\log_{1/p}(n) \cdot p(h = k)
= \log_{1/p}(n) \cdot \sum\limits_{k = 1}^{\log_{1/p}(n)} p(h = k)
= O(\log n)
$

Вторая сумма:
$ \sum\limits_{\log_{1/p}(n) + 1}^{\infty}k \cdot p(h = k)
< \sum\limits_{\log_{1/p}(n) + 1}^{\infty}k \cdot np^k 
< \sum\limits_{\log_{1/p}(n)}^{\infty}k \cdot np^k
= n \cdot \sum\limits_{\log_{1/p}(n)}^{\infty}k p^k
= n p^{\log_{1/p} n} \cdot \sum\limits_0^{\infty} (k + \log_{1/p} n) \cdot p^k
= n p^{\log_{1/p} n} \cdot (\sum\limits_0^{\infty} k p^k + \log_{1/p} n \sum\limits_0^{\infty} p^k)
= n p^{\log_{1/p} n} \cdot (O(1) + \log_{1/p} n \cdot O(1))
= O(\log n)
$

Cледственно, **skip quardtree** в среднем содержит $O(\log n)$ уровней.

### Оценка памяти

Для хранения **skip quardtree** необходимо $O(n)$ памяти

**Док-во:**
Сжатое квадродерево для $n$ точек занимает $O(n)$ памяти. На нулевом уровне $n$ точек. На следующем уровне $O(p n)$ точек, дальше $O(p^2 n)$ и так далее.
В итоге получаем $\sum\limits_0^{O(\log n)} O(p^k n) < \sum\limits_0^{\infty} O(p^k n) = O(n) $.